STEP 1 – Install and Import Libraries

👉 Create a new cell and paste this, then run it:

In [ ]:
import glob
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
import numpy as np

print("TF version:", tf.__version__)


Define Image Size and Simple Lane Model

We’ll build a simple MobileNetV2 + upsampling model (MobileNetV3 can be version-dependent, so this is safer and still correct for your project).

In [ ]:
# 1) Define model (fixed upsampling)
# -------------------
IMG_HEIGHT = 256
IMG_WIDTH = 256
IMG_CHANNELS = 3

def build_lane_model():
    base_model = tf.keras.applications.MobileNetV2(
        input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS),
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False  # freeze backbone

    inputs = layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))

    # Normalization
    x = layers.Rescaling(1./255)(inputs)

    # Backbone (this will reduce 256x256 -> 8x8 feature map)
    x = base_model(x)

    # Simple SCNN-like block
    def spatial_block(t):
        h = layers.Conv2D(64, (1, 3), padding='same', activation='relu')(t)
        v = layers.Conv2D(64, (3, 1), padding='same', activation='relu')(t)
        return layers.Concatenate()([t, h, v])

    x = spatial_block(x)

    # Decoder: UpSample back to 256x256
    # 8x8 -> 16x16
    x = layers.Conv2D(128, 3, padding='same', activation='relu')(x)
    x = layers.UpSampling2D(size=(2, 2))(x)

    # 16x16 -> 32x32
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.UpSampling2D(size=(2, 2))(x)

    # 32x32 -> 64x64
    x = layers.Conv2D(32, 3, padding='same', activation='relu')(x)
    x = layers.UpSampling2D(size=(2, 2))(x)

    # 64x64 -> 128x128
    x = layers.Conv2D(16, 3, padding='same', activation='relu')(x)
    x = layers.UpSampling2D(size=(2, 2))(x)

    # 128x128 -> 256x256
    x = layers.Conv2D(8, 3, padding='same', activation='relu')(x)
    x = layers.UpSampling2D(size=(2, 2))(x)

    # Final 1-channel mask: 256x256x1
    outputs = layers.Conv2D(1, 1, activation='sigmoid')(x)

    model = models.Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

model = build_lane_model()
model.summary()


Make a Dummy “Fake” Dataset (Just to Test Training)

Right now you don’t yet have lane images loaded, so we’ll create dummy data.
This is just to check that training runs and code is correct.

In [ ]:
# 2) Create dummy data with 256x256 masks
# -------------------
NUM_SAMPLES = 10

X_dummy = np.random.randint(
    0, 256, size=(NUM_SAMPLES, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
).astype('float32')

Y_dummy = np.random.randint(
    0, 2, size=(NUM_SAMPLES, IMG_HEIGHT, IMG_WIDTH, 1)
).astype('float32')

print("X_dummy shape:", X_dummy.shape)
print("Y_dummy shape:", Y_dummy.shape)



Train the Model (Just a Quick Test)

In [ ]:

# 3) Train for 2 epochs
# -------------------
print("\nStarting training...")
history = model.fit(
    X_dummy, Y_dummy,
    epochs=2,
    batch_size=2,
    verbose=1
)

print("\nEpochs actually trained:", len(history.history['loss']))


Test Prediction and Visualize (For Understanding)

In [ ]:
# Take one dummy image
test_img = X_dummy[0:1]  # shape (1, H, W, 3)
pred_mask = model.predict(test_img)[0, ..., 0]  # (H, W)

plt.figure(figsize=(10,4))
plt.subplot(1,2,1); plt.title("Input image"); plt.imshow(test_img[0].astype('uint8')); plt.axis("off")
plt.subplot(1,2,2); plt.title("Predicted mask"); plt.imshow(pred_mask, cmap='gray'); plt.axis("off")
plt.show()

making a folder

In [ ]:
!mkdir -p /content/my_lanes/images
!mkdir -p /content/my_lanes/masks


Replace the earlier X_dummy / Y_dummy with this loader:

In [ ]:
import numpy as np

IMG_HEIGHT = 256
IMG_WIDTH = 256

image_paths = sorted(glob.glob("/content/my_lanes/images/*"))
mask_paths  = sorted(glob.glob("/content/my_lanes/masks/*"))

print("Images:", len(image_paths), "Masks:", len(mask_paths))

X_real, Y_real = [], []

for img_p, mask_p in zip(image_paths, mask_paths):
    img = cv2.imread(img_p)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))

    mask = cv2.imread(mask_p, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (IMG_WIDTH, IMG_HEIGHT))
    mask = (mask > 128).astype("float32")

    X_real.append(img)
    Y_real.append(mask[..., None])

X_real = np.array(X_real, dtype="float32")
Y_real = np.array(Y_real, dtype="float32")

print("X_real shape:", X_real.shape)
print("Y_real shape:", Y_real.shape)


STEP 8 – Train on Real Data

Now train your existing model (no change needed):

In [ ]:
history = model.fit(
    X_real, Y_real,
    epochs=20,
    batch_size=2
)


Take a input from folder for prediction

STEP 10 – See Real Output

Use the same visualization code as before:

# Task
Train the lane detection model using images and masks from '/content/my_lanes/images' and '/content/my_lanes/masks'. Then, create a new folder `/content/my_lanes/predict_input` for input images, load images from it, predict their corresponding lane masks, and save these predicted masks to `/content/my_lanes/predicted_masks`. Finally, visualize a few input images with their generated masks.

# Task
Train the lane detection model using images and masks from `/content/my_lanes/images` and `/content/my_lanes/masks`. Then, create a new folder `/content/my_lanes/predict_input` for input images, load images from it, predict their corresponding lane masks, and save these predicted masks to `/content/my_lanes/predicted_masks`. Finally, visualize a few input images with their generated masks.

## Create Input Prediction Folder

### Subtask:
Create a new directory named `/content/my_lanes/predict_input` where you will upload the images you want the model to predict on. You will need to manually upload images to this folder after it's created.


**Reasoning**:
The subtask requires creating a new directory named `/content/my_lanes/predict_input`. I will use the `!mkdir -p` shell command in a code cell to create this directory.



In [ ]:
!mkdir -p /content/my_lanes/predict_input

After the `/content/my_lanes/predict_input` directory has been created, please manually upload the images you wish to predict on into this folder. These images should be in a common format such as PNG or JPG. Once the images are uploaded, you can proceed with the next steps of the task.

After the `/content/my_lanes/predict_input` directory has been created, please manually upload the images you wish to predict on into this folder. These images should be in a common format such as PNG or JPG. Once the images are uploaded, you can proceed with the next steps of the task.

**Reasoning**:
The subtask requires loading images from the newly created `/content/my_lanes/predict_input` directory, which is the first step towards predicting lane masks. This code block will also prepare for saving predicted masks by creating the output directory.



In [ ]:
import glob
import cv2
import numpy as np
import os

# Create a directory for predicted masks
!mkdir -p /content/my_lanes/predicted_masks

# Define image dimensions (already defined, but good to ensure context)
IMG_HEIGHT = 256
IMG_WIDTH = 256

# Get paths of input images for prediction
predict_image_paths = sorted(glob.glob("/content/my_lanes/predict_input/*"))

print(f"Found {len(predict_image_paths)} images for prediction.")

X_predict = []

for img_p in predict_image_paths:
    img = cv2.imread(img_p)
    if img is None:
        print(f"Warning: Could not read image {img_p}. Skipping.")
        continue
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    X_predict.append(img)

if not X_predict:
    print("No images were loaded for prediction. Please upload images to /content/my_lanes/predict_input and run this cell again.")
    X_predict = np.array([], dtype="float32").reshape(0, IMG_HEIGHT, IMG_WIDTH, 3)
else:
    X_predict = np.array(X_predict, dtype="float32")

print("X_predict shape:", X_predict.shape)


**Reasoning**:
Now that the input images for prediction are loaded into `X_predict`, the next step is to use the trained `model` to predict the lane masks for these images. These predicted masks then need to be saved to the previously created `/content/my_lanes/predicted_masks` directory.



In [ ]:
print("Starting prediction...")

# Predict masks for the loaded images
predicted_masks = model.predict(X_predict)

print("Predicted masks shape:", predicted_masks.shape)

# Save the predicted masks
for i, mask in enumerate(predicted_masks):
    # Convert mask to 0-255 uint8 format
    mask_img = (mask * 255).astype(np.uint8)

    # Get original image filename to use for saving mask
    original_filename = os.path.basename(predict_image_paths[i])
    mask_filename = f"predicted_mask_{original_filename}"
    save_path = os.path.join("/content/my_lanes/predicted_masks", mask_filename)

    cv2.imwrite(save_path, mask_img)

print(f"Saved {len(predicted_masks)} predicted masks to /content/my_lanes/predicted_masks")

**Reasoning**:
The subtask requires adjusting the plot size and subplot arrangement to display input images and their corresponding predicted masks side-by-side, with each pair in a new row, for better visibility. I will modify the existing plotting code to implement these changes, including adjusting `num_to_visualize`, `figsize`, and the `plt.subplot()` calls as specified in the instructions.



In [ ]:
num_to_visualize = 15 # Number of image-mask pairs to visualize. Changed for better visibility.

plt.figure(figsize=(6, num_to_visualize * 3)) # Adjusted figsize for new layout

for i in range(num_to_visualize):
    # Display original image
    plt.subplot(num_to_visualize, 2, 2*i + 1) # Arrange in num_to_visualize rows, 2 columns
    plt.title(f"Input Image {i+1}")
    plt.imshow(X_predict[i].astype('uint8'))
    plt.axis("off")

    # Display predicted mask
    plt.subplot(num_to_visualize, 2, 2*i + 2) # Arrange in num_to_visualize rows, 2 columns
    plt.title(f"Predicted Mask {i+1}")
    plt.imshow(predicted_masks[i, ..., 0], cmap='gray')
    plt.axis("off")

plt.tight_layout()
plt.show()